# Code using linescore api

In [75]:
import requests
import datetime

def get_current_game_info(team_id):
    games_url = 'http://statsapi.mlb.com/api/v1/schedule/games/?sportId=1'
    result = requests.get(games_url)
    games = result.json()['dates'][0]['games']

    # Get current date in UTC. Will this work for days when a team plays a doubleheader?
    current_date_utc = datetime.datetime.utcnow()

    for game in games:
        home_team_id = game['teams']['home']['team']['id']
        away_team_id = game['teams']['away']['team']['id']
        
        game_date = datetime.datetime.strptime(game['gameDate'], "%Y-%m-%dT%H:%M:%SZ")

        if (home_team_id == team_id or away_team_id == team_id) and game_date.date() == current_date_utc.date():
            home_team_name = game['teams']['home']['team']['name']
            away_team_name = game['teams']['away']['team']['name']
            game_pk = game['gamePk']
            return game_pk, home_team_name, away_team_name

    # Return None if no game is found
    return None, None, None

# ID for Philadelphia Phillies
phillies_id = 143

game_id, home_team, away_team = get_current_game_info(phillies_id)

if game_id is not None:
    print(f"Game ID for current Phillies game: {game_id}")
    print(f"Home Team: {home_team}")
    print(f"Away Team: {away_team}")
else:
    print("No game found for the Philadelphia Phillies today.")

Game ID for current Phillies game: 717408
Home Team: Philadelphia Phillies
Away Team: San Diego Padres


In [98]:
import requests

def get_game_info(api_url):
    response = requests.get(api_url)
    json_data = response.json()

    game_info = {}

    # get list of innings from json
    innings = json_data.get('innings', [])

    # iterate over each inning
    for inning in innings:
        inning_num = inning['num']

        # get the home and away runs for each inning
        home_runs = inning['home'].get('runs', 0)
        away_runs = inning['away'].get('runs', 0)

        game_info[inning_num] = {
            'home_runs': home_runs,
            'away_runs': away_runs,
        }

    # General game info
    game_info['general'] = {
        'balls': json_data.get('balls', 0),
        'strikes': json_data.get('strikes', 0),
        'outs': json_data.get('outs', 0),
    }

    # Placeholder for who's on base (as this information may not be available from the API)
    game_info['onBase'] = {
        'first': "N/A",
        'second': "N/A",
        'third': "N/A",
    }

    # Total runs, hits, and errors for each team
    game_info['total'] = {
        'home': {
            'runs': json_data.get('teams', {}).get('home', {}).get('runs', 0),
            'hits': json_data.get('teams', {}).get('home', {}).get('hits', 0),
            'errors': json_data.get('teams', {}).get('home', {}).get('errors', 0),
        },
        'away': {
            'runs': json_data.get('teams', {}).get('away', {}).get('runs', 0),
            'hits': json_data.get('teams', {}).get('away', {}).get('hits', 0),
            'errors': json_data.get('teams', {}).get('away', {}).get('errors', 0),
        }
    }

    return game_info


# Example usage:
api_url = f'https://statsapi.mlb.com/api/v1/game/{game_id}/linescore'.format(game_id=game_id)
game_info = get_game_info(api_url)

game_info

{1: {'home_runs': 1, 'away_runs': 0},
 2: {'home_runs': 1, 'away_runs': 0},
 3: {'home_runs': 0, 'away_runs': 0},
 4: {'home_runs': 0, 'away_runs': 0},
 5: {'home_runs': 0, 'away_runs': 0},
 'general': {'balls': 0, 'strikes': 0, 'outs': 2},
 'onBase': {'first': 'N/A', 'second': 'N/A', 'third': 'N/A'},
 'total': {'home': {'runs': 2, 'hits': 4, 'errors': 0},
  'away': {'runs': 0, 'hits': 5, 'errors': 1}}}

In [89]:
def format_game_info(game_info):
    formatted_info = ""

    # Print general game info
    formatted_info += "\nGeneral Game Info:\n"
    formatted_info += f"\tBalls: {game_info['general']['balls']}\n"
    formatted_info += f"\tStrikes: {game_info['general']['strikes']}\n"
    formatted_info += f"\tOuts: {game_info['general']['outs']}\n"

    # Print info about who's on base
    formatted_info += "\nBases Occupied:\n"
    for base, occupied in game_info['onBase'].items():
        formatted_info += f"\tBase {base.capitalize()}: {'Yes' if occupied else 'No'}\n"

    # Print total game info
    formatted_info += "\nTotal Game Info:\n"
    for team, team_info in game_info['total'].items():
        formatted_info += f"\t{team.capitalize()} Team:\n"
        formatted_info += f"\t\tRuns: {team_info['runs']}\n"
        formatted_info += f"\t\tHits: {team_info['hits']}\n"
        formatted_info += f"\t\tErrors: {team_info['errors']}\n"

    return formatted_info

# Example usage:
print(format_game_info(game_info))



General Game Info:
	Balls: 0
	Strikes: 0
	Outs: 0

Bases Occupied:
	Base First: Yes
	Base Second: Yes
	Base Third: Yes

Total Game Info:
	Home Team:
		Runs: 2
		Hits: 4
		Errors: 0
	Away Team:
		Runs: 0
		Hits: 4
		Errors: 1



In [100]:
import time
import os
game_id = 717402
api_url = f'https://statsapi.mlb.com/api/v1/game/{game_id}/linescore'.format(game_id=game_id)

game_info = get_game_info(api_url)
print(api_url)
print(format_game_info(game_info))




https://statsapi.mlb.com/api/v1/game/717402/linescore

General Game Info:
	Balls: 2
	Strikes: 1
	Outs: 2

Bases Occupied:
	Base First: Yes
	Base Second: Yes
	Base Third: Yes

Total Game Info:
	Home Team:
		Runs: 2
		Hits: 4
		Errors: 0
	Away Team:
		Runs: 0
		Hits: 5
		Errors: 1



# Code using live feed api

In [116]:
# Find all games that are currently in progress

import requests
import json

def get_data_from_api(url):
    response = requests.get(url)
    data = json.loads(response.text)
    return data

def get_games_happening_now(data):
    games_now = []
    for date in data['dates']:
        for game in date['games']:
            if game['status']['abstractGameState'] == 'In Progress' or game['status']['detailedState'] == 'In Progress':
                games_now.append({
                    'gamePk': game['gamePk'],
                    'home_team': game['teams']['home']['team']['name'],
                    'away_team': game['teams']['away']['team']['name']
                })
    return games_now

def run(url):
    data = get_data_from_api(url)
    games_now = get_games_happening_now(data)
    return games_now

# Usage
url = 'https://statsapi.mlb.com/api/v1/schedule/games/?sportId=1'
games_now = run(url)
for game in games_now:
    print(game)


{'gamePk': 717381, 'home_team': 'Los Angeles Angels', 'away_team': 'Houston Astros'}


In [194]:
import requests
import json

def get_data_from_api(url):
    response = requests.get(url)
    data = json.loads(response.text)
    return data

def get_inning_scores(data):
    innings = data['liveData']['linescore']['innings']
    scores = []
    for inning in innings:
        scores.append({
            'inning': inning['num'],
            'home_runs': inning['home'].get('runs'),
            'away_runs': inning['away'].get('runs')
        })
    return scores

def get_team_stats(data):
    teams = data['liveData']['linescore']['teams']
    return {
        'home': {
            'runs': teams['home']['runs'],
            'hits': teams['home']['hits'],
            'errors': teams['home']['errors']
        },
        'away': {
            'runs': teams['away']['runs'],
            'hits': teams['away']['hits'],
            'errors': teams['away']['errors']
        }
    }

def get_current_count(data):
    linescore = data['liveData']['linescore']
    return {
        'balls': linescore['balls'],
        'strikes': linescore['strikes'],
        'outs': linescore['outs']
    }

def get_base_runners(data):
    bases = { '1B': False, '2B': False, '3B': False }
    current_play = data['liveData']['plays']['currentPlay']

    # Check for runners on bases
    if 'postOnFirst' in current_play['matchup']:
        bases['1B'] = True
    if 'postOnSecond' in current_play['matchup']:
        bases['2B'] = True
    if 'postOnThird' in current_play['matchup']:
        bases['3B'] = True

    return bases

def run(game_id):
    api_url = f'https://statsapi.mlb.com/api/v1.1/game/{game_id}/feed/live'
    data = get_data_from_api(api_url)
    inning_scores = get_inning_scores(data)
    team_stats = get_team_stats(data)
    current_count = get_current_count(data)
    base_runners = get_base_runners(data)
    return {
        'inning_scores': inning_scores,
        'team_stats': team_stats,
        'current_count': current_count,
        'base_runners': base_runners
    }

def print_game_info(game_info):
    print("Inning Scores:")
    for score in game_info['inning_scores']:
        print(f"Inning {score['inning']}: Home: {score['home_runs']} Away: {score['away_runs']}")
    print("Team Stats:")
    print(f"Home: Runs: {game_info['team_stats']['home']['runs']} Hits: {game_info['team_stats']['home']['hits']} Errors: {game_info['team_stats']['home']['errors']}")
    print(f"Away: Runs: {game_info['team_stats']['away']['runs']} Hits: {game_info['team_stats']['away']['hits']} Errors: {game_info['team_stats']['away']['errors']}")
    print("Current Count:")
    print(f"Balls: {game_info['current_count']['balls']} Strikes: {game_info['current_count']['strikes']} Outs: {game_info['current_count']['outs']}")
    print("Base Runners:")
    print(f"1B: {game_info['base_runners']['1B']} 2B: {game_info['base_runners']['2B']} 3B: {game_info['base_runners']['3B']}")


In [195]:
# Usage
game_id = 717381  # replace with your game ID
results = run(game_id)
print_game_info(results)

Inning Scores:
Inning 1: Home: 0 Away: 1
Inning 2: Home: 1 Away: 0
Inning 3: Home: 0 Away: 0
Inning 4: Home: 2 Away: 0
Inning 5: Home: 0 Away: 0
Inning 6: Home: 0 Away: 0
Team Stats:
Home: Runs: 3 Hits: 4 Errors: 0
Away: Runs: 1 Hits: 9 Errors: 0
Current Count:
Balls: 1 Strikes: 2 Outs: 3
Base Runners:
1B: False 2B: False 3B: False


# Endpoint documentation

See this page for more info on MLB API endpoints (e.g. how to call linescore)

https://github.com/toddrob99/MLB-StatsAPI/blob/master/statsapi/endpoints.py